<br>

# Introdução

In [ ]:
import json
import os
import random
import re
import time

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from bs4 import BeautifulSoup
from folium import plugins
from osgeo import gdal, osr
from tqdm.notebook import tqdm, trange

<br>

# Join Data

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_eda/main/data/tabs/tab_municipio_eda.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

<br>

## Create pop up's column

In [ ]:
# Add Field
def popup_html(row):
    tel1 = str(row['tel_1']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel2 = str(row['tel_2']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    fax1 = str(row['fax_1']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    fax2 = str(row['fax_2']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}<br>
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['unidaderegional']) else '{}'.format(row['unidaderegional']),
        '' if pd.isnull(row['endereco'])       else '{}'.format(row['endereco']),
        #'' if pd.isnull(row['numero'])         else ', {}'.format(row['numero']),
        #'' if pd.isnull(row['complemento'])    else ' - {}'.format(row['complemento']),
        #'' if pd.isnull(row['bairro'])         else 'Bairro: {}<br>'.format(row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else 'Município: {}<br>'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep'])            else 'CEP: {}<br>'.format(row['cep']),
        # Contatos
        '' if pd.isnull(row['tel_1'])          else 'Telefone: <a href="tel:{}">{}</a><br>'.format(tel1, row['tel_1']),
        '' if pd.isnull(row['fax_1'])            else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax1, row['fax_1']),
        '' if pd.isnull(row['email'])          else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(row['email'])          else '<a href="{}" target="_blank">Conferir Dados</a>'.format(row['url']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
# Calculate PopUps
gdf['popup'] = gdf.apply(popup_html, axis=1)

<br>

## Adjust Table

In [ ]:
# Delete Columns
gdf.drop([
    'id_municipio', 'endereco',
    #'numero', 'bairro',
    'municipio_sede', 'cep', 'tel_1', 'fax_1',
    #'complemento', 
    'url', 'email',], axis=1, inplace=True)
print(gdf.columns)

In [ ]:
# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_eda.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

## Get Centroid

In [ ]:
def get_centroid(gdf):
    gdf['apenasparacentroid'] = 35
    gdf_dissolve = gdf.dissolve(by='apenasparacentroid')
    gdf_centroid = gdf_dissolve.representative_point()
    gdf = gdf.drop('apenasparacentroid', axis=1)
    return [float(gdf_centroid.y), float(gdf_centroid.x)]

In [ ]:
list_centroid = get_centroid(gdf)
list_centroid

## Folium

In [ ]:
def map_bomb(input_geojson, bbox):
    gdf = gpd.read_file(input_geojson)
    
    # Column with category
    col_categories = 'unidaderegional'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    #color_polygon['Agência Ambiental de Ribeirão Preto'] = '#e699ff'
    #color_polygon['Agência Ambiental de Barretos'] = '#4da6ff'
    #color_polygon['Agência Ambiental de São Carlos'] = '#ff5050'
    #color_polygon['Agência Ambiental de Osasco'] = '#00264d'

    # Create Map
    m = folium.Map(
        location=[-22.545968889465207, -49.56185387118866],
        zoom_start=6,
        min_zoom=6,
        max_zoom=11,
        max_bounds=True,
        min_lon = bbox['min_lon']*(101/100),
        max_lon = bbox['max_lon']*(99/100),        
        min_lat = bbox['min_lat']*(101/100),
        max_lat = bbox['max_lat']*(99/100),
        #zoom_delta=0.1,
    )
    folium.GeoJson(
        gdf,
        name='EDA',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'unidaderegional'],
            aliases=['Munícipio', 'Unidade EDA'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )        
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
bbox = {
    'max_lat': 0,
    'max_lon': 0,
    'min_lat': 0,
    'min_lon': 0,
}
m = map_bomb(os.path.join('data', 'shps', 'sp_eda.geojson'), bbox)

# Map with Bounds
bbox = {
    'max_lat': m.get_bounds()[1][0],
    'min_lat': m.get_bounds()[0][0],
    'max_lon': m.get_bounds()[1][1],
    'min_lon': m.get_bounds()[0][1],
}
m = map_bomb(os.path.join('data', 'shps', 'sp_eda.geojson'), bbox)

# Results
print(bbox)
m

<br>

# Save Map

In [ ]:
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'eda_map.html'))
m.save(os.path.join('..', '..', 'case_django', 'divadmin', 'templates', 'eda_map.html'))